In [1]:
import numpy as np

In [2]:
from braket.aws import AwsDevice, AwsQuantumTask
from braket.circuits import Circuit
from braket.devices import LocalSimulator 
from braket.tracking import Tracker

# AWS local simulator

#device = LocalSimulator()

# AWS online simulator (not free)
#device = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/sv1")

# DM1 Simulator (adds noise)
device = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/dm1")

# Lucy
#device = AwsDevice("arn:aws:braket:eu-west-2::device/qpu/oqc/Lucy")

In [10]:
from core import TwoBitCircuit_noCode, ThreeBitCircuit_noCode, FourBitCircuit_noCode, FiveBitCircuit_noCode, SixBitCircuit_noCode
from braket.circuits import Circuit, noises, Gate

#Data collection for dependence of Shannon Entropy and Minimum Entropy on the error rate for quantum system.
shotsnum = 5000
cq = 0
aq = 1

TwoCircuit = TwoBitCircuit_noCode(cq,aq).circuit
ThreeCircuit = TwoBitCircuit_noCode(cq, aq).circuit
FourCircuit = TwoBitCircuit_noCode(cq, aq).circuit
FiveCircuit = TwoBitCircuit_noCode(cq, aq).circuit
SixCircuit = TwoBitCircuit_noCode(cq, aq).circuit 
circuits = [TwoCircuit, ThreeCircuit, FourCircuit, FiveCircuit, SixCircuit]

In [11]:
#Introducing error through the AWS Braket SDK. Operates on quantum states through quantum channels essentially applying a projection of the 
#density matrix. Use depolarizing error because it represents information loss due to coupling with environment and represents the 
#Github method of applying superposition of pauli matrices to the states to introduce error. 
noise1 = [noises.Depolarizing(probability=0.05*(1+i)) for i in range(0, 5)]
noise2 = [noises.BitFlip(probability=0.05*(1+i)) for i in range(0, 5)]

for i in range(0, 5):
    circuits[i].apply_gate_noise(noise1[i])
    circuits[i].apply_readout_noise(noise2[i])

In [12]:
with Tracker() as tracker:
    Two_task = device.run(TwoCircuit, shots=shotsnum)
    Three_task = device.run(ThreeCircuit, shots=shotsnum)
    Four_task = device.run(FourCircuit, shots=shotsnum)
    Five_task = device.run(FiveCircuit, shots=shotsnum)
    Six_task = device.run(SixCircuit, shots=shotsnum)

In [13]:
print(tracker.quantum_tasks_statistics())
print(tracker.simulator_tasks_cost())
print(tracker.qpu_tasks_cost())
print(Two_task.id)
print(Three_task.id)
print(Four_task.id)
print(Five_task.id)
print(Six_task.id)

{'arn:aws:braket:::device/quantum-simulator/amazon/dm1': {'shots': 25000, 'tasks': {'CREATED': 5}}}
0
0
arn:aws:braket:us-east-1:592242689881:quantum-task/5a569e26-886c-4f84-a498-27578e38ec88
arn:aws:braket:us-east-1:592242689881:quantum-task/ce7b6132-a667-4b80-b4be-1fea51c237dc
arn:aws:braket:us-east-1:592242689881:quantum-task/ec7ebf1d-c771-45c8-9147-e93d24cdcfb9
arn:aws:braket:us-east-1:592242689881:quantum-task/b01feeb5-f44f-4f0a-8ad2-8bc1fa1f014c
arn:aws:braket:us-east-1:592242689881:quantum-task/8e29a3a5-d1e0-459c-9439-193055f8eb16


In [14]:
Two_prob = AwsQuantumTask(arn = Two_task.id).result().measurement_probabilities
Three_prob = AwsQuantumTask(arn = Three_task.id).result().measurement_probabilities
Four_prob = AwsQuantumTask(arn = Four_task.id).result().measurement_probabilities
Five_prob = AwsQuantumTask(arn = Five_task.id).result().measurement_probabilities
Six_prob = AwsQuantumTask(arn = Six_task.id).result().measurement_probabilities
print(Two_prob)
print(Three_prob)
print(Four_prob)
print(Five_prob)
print(Six_prob)

{'00': 0.4146, '10': 0.0802, '01': 0.0706, '11': 0.4346}
{'11': 0.3654, '00': 0.3772, '01': 0.131, '10': 0.1264}
{'00': 0.3252, '01': 0.169, '11': 0.3292, '10': 0.1766}
{'00': 0.3038, '11': 0.2924, '01': 0.209, '10': 0.1948}
{'11': 0.2786, '01': 0.2256, '10': 0.2192, '00': 0.2766}


In [15]:
Two_raw = AwsQuantumTask(arn = Two_task.id).result().measurements
Three_raw = AwsQuantumTask(arn = Three_task.id).result().measurements
Four_raw = AwsQuantumTask(arn = Four_task.id).result().measurements
Five_raw = AwsQuantumTask(arn = Five_task.id).result().measurements
Six_raw = AwsQuantumTask(arn = Six_task.id).result().measurements
np.savetxt('braketError_05_raw.txt',Two_raw,fmt="%i")
np.savetxt('braketError_10_raw.txt',Three_raw,fmt="%i")
np.savetxt('braketError_15_raw.txt',Four_raw,fmt="%i")
np.savetxt('braketError_20_raw.txt',Five_raw,fmt="%i")
np.savetxt('braketError_25_raw.txt',Six_raw,fmt="%i")